![header](http://eurogoos.eu/wp-content/uploads/SOCIB-logo.jpg)

# SOCIB API TRAINING
<div style="text-align: right"><i> 06-Part-one-out-of-06 </i></div>

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span>
    </li>
    <li><span><a href="#Setup" data-toc-modified-id="Setup-2">
        <span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span>
        <ul class="toc-item">
            <li>
                <span><a href="#Importing-modules" data-toc-modified-id="Importing-modules-2.1">
                    <span class="toc-item-num">2.1&nbsp;&nbsp;</span>Importing modules</a></span>
            </li>
            <li>
                <span><a href="#API-token" data-toc-modified-id="API-token-2.2">
                    <span class="toc-item-num">2.2&nbsp;&nbsp;</span>API token</a></span>
            </li>
            <li>
                <span><a href="#Auxiliary-functions" data-toc-modified-id="Auxiliary-functions-2.3">
                    <span class="toc-item-num">2.3&nbsp;&nbsp;</span>Auxiliary functions</a></span>
            </li>
        </ul>
    <li>
        <span><a href="#Resampling-a-given-entry/file-content-(data!)-with-SOCIB-API:" data-toc-modified-id="Resampling-a-given-entry/file-content-(data!)-with-SOCIB-API:"><span class="toc-item-num">3&nbsp;&nbsp;</span>Resampling a given entry/file content (data!) with SOCIB API:</a></span>
        <ul><li><span><a href="#Upsampling/Downsampling" data-toc-modified-id="Upsampling/Downsampling"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Upsampling/Downsampling</a></span></li>
        </ul>
     </li>
    <li><span><a href="#Next-tutorial" data-toc-modified-id="Next-tutorial">
        <span class="toc-item-num">4&nbsp;&nbsp;</span>Next tutorial</a></span></li>
    </ul>
</div>

---

# ENTRIES

## Introduction 

SOCIB API is a door users can knock-on in order to get information about the Balearic Islands Coastal Ocean Observing and Forecast System (SOCIB). SOCIB API is represented by an generic url (SOCIB API url). The elements that trigger a response when added to the generic API url are called `ENDPOINTS`.
Among the present posibilities:
<ul>
    <li>measured variables (<span class="alert-info">/standard-variables/</span>)</li>
    <li>stock of instruments (<span class="alert-info">/instrument-types/</span>) and platforms(<span class="alert-info">/platform-types/</span>), 
</li>
    <li>data maturity (<span class="alert-info">/processing-levels/</span> and <span class="alert-info">/data-modes/</span>)</li>
    <li>kind of data (<span class="alert-info">/feature-types/</span>)</li>
    <li>data entities (<span class="alert-success"><b>/entries/<b></span>, <span class="alert-info">/data-sources/</span>, <span class="alert-info">/instruments/</span>,<span class="alert-info">/platforms/</span>, <span class="alert-info">/data-products/</span>)</li>

</ul>

<br>This notebook will focus then on the <span class="alert-success" style=""><b>/entries/</b></span> endpoint.

---

## Setup

### Importing modules

We will relly on a set of python modules to deal with SOCIB API next.<br> `Please run the next cells` so that they can be used by the present Jupyter Notebook:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
import pandas
import requests
import IPython
from json2html import *

In [ ]:
import matplotlib.pyplot as plt
import datetime

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If any of them raises any error please run prior to load it and in a dedicated cell, either:<ul><li> <i>`!conda install packageName --yes`</i></li> or <li><i>`!pip install packageName --yes`</i></li></ul>

### API token

To be able to query SOCIB API you will need first a <i>token</i> (api key).<br>To get one please visit the [API home page](http://api.socib.es/home/) and fill-in the form at bottom. An email will be sent to you with such <i>token</i>.

`Please run the next cell if you wanna have a glimpse to API home page`:

In [ ]:
IPython.display.HTML('<iframe src="http://api.socib.es/home/"" width=100% height=500></iframe>')

`Please set in the next cell your api_key and run the cell below to load it in memory for later use`:

In [ ]:
api_key = '' #write here the token emailed to you

In [ ]:
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If you do not remember your token or wanna ask for a new one please reach <i>data.center@socib.es</i> with the following `subject`: 'SOCIB API TOKEN: UPDATE/REMIND REQUEST'

### Auxiliary functions

In [ ]:
def timeSeriePlot(feature, param_name,label,title):
    data = {}
    data[label] = [variable for variable in feature[0]['variables'] if variable['param_name']==param_name][0]['data']
    times = [timestamp for timestamp in feature[0]['coordinates']['time']['data']]
    df = pandas.DataFrame(data=data,index=times)
    ax = df.plot(grid=True,figsize=(12,5), title=title, marker='o', linestyle='-')
    ax.set_xlabel("TIME") , ax.set_ylabel(label)
    plt.xticks(rotation=45,horizontalalignment='right',fontweight='light',fontsize='medium')

---

## Resampling a given entry/file content (data!) with SOCIB API:

As we already saw in [05-entry-data.ipynb](05-entry-data.ipynb) SOCIB API enables to perform on-the-fly subsetting operations. Nevertheless, is it possible that users not only want to subset the data but also resampling: this is, to retrieve more or less values from the original set that matches the configured sampling interval of the instrument.

Let's use a sea water temperature Time Serie data to illustrate these operations:; ie from one of SOICB ocenographic buoys: Buoy_BahiaDePalma

In [ ]:
platform_id = 'Buoy_BahiaDePalma'
feature_type = 'TimeSeries'
standard_variable = 'air_pressure'

In [ ]:
end_point = '/entries/'
query_parameters = 'platform='+platform_id+'&feature_type='+feature_type+'&standard_variable='+standard_variable
url = '%s%s?%s' % (api_url, end_point, query_parameters)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)

print('API custom query : '+ url)
print('There are %s netCDFs/files produced involving such platform...'%(entries_response['count']))
print('Find next a preview of the first %s ones'%(len(entries_response['results'])))
pandas.DataFrame.from_dict(entries_response['results'])

Let's work onwards with the first Time Serie:

In [ ]:
entry_id = '2587f63c89'

In [ ]:
end_point = '/entries/'+entry_id
aux_endpoint = '/data/'
query_parameters = 'standard_variable='+standard_variable
url = '%s%s%s?%s' % (api_url, end_point, aux_endpoint, query_parameters)
entry_data_request= requests.get(url, headers=headers)
entry_data_response = json.loads(entry_data_request.text)

print('API query: '+ url)
print('Find next the preview:')
pandas.DataFrame.from_dict(entry_data_response)

In [ ]:
pandas.DataFrame.from_dict(entry_data_response[0]['variables'])

Let's plot some data!:

In [ ]:
param_name, label = 'AIRP', 'AIR PRESSURE'
title = label+' ('+param_name+') from '+ platform_id

In [ ]:
timeSeriePlot(entry_data_response,param_name,label,title)

Let's focus on the last 7 days of data:

In [ ]:
today = datetime.datetime.strptime(times[-1],'%Y-%m-%dT%H:%M:%S')
oneweekago = (today - datetime.timedelta(days=7)).strftime('%Y-%m-%dT%H:%M:%S')
oneweekago

In [ ]:
end_point = '/entries/'+entry_id
aux_endpoint = '/data/'
query_parameters = 'standard_variable='+standard_variable+'&initial_datetime='+oneweekago+'&end_datetime='+times[-1]
url = '%s%s%s?%s' % (api_url, end_point, aux_endpoint, query_parameters)
entry_data_time_range_subset_request= requests.get(url, headers=headers)
entry_data_time_range_subset_response = json.loads(entry_data_time_range_subset_request.text)

print('API query: '+ url)
print('Find next the preview:')
pandas.DataFrame.from_dict(entry_data_time_range_subset_response)

In [ ]:
timeSeriePlot(entry_data_time_range_subset_response,param_name,label,title)

#### Upsampling/Downsampling

Let's image you want to either reduce (downsampling) or increment (upsampling) the number of measurements originally provided (matching the instrument sampling interval). SOCIB API enables this kind of operations by means of its `resample_how` and `resample_rule` and `platform` query parameters.
<br>Here after some examples about how-to-use them: <br>

<ul><li>Upsampling</li></ul>

Let's image I want data every minute instead!:

In [ ]:
how, rule = 'mean', '1min'

In [ ]:
end_point = '/entries/'+entry_id
aux_endpoint = '/data/'
query_parameters = 'resample_how='+how+'&resample_rule='+rule+'&standard_variable='+standard_variable+'&initial_datetime='+oneweekago+'&end_datetime='+times[-1]
url = '%s%s%s?%s' % (api_url, end_point, aux_endpoint, query_parameters)
entry_data_upsampling_request= requests.get(url, headers=headers)
entry_data_upsampling_response = json.loads(entry_data_upsampling_request.text)

print('API query: '+ url)
print('Find next the preview:')
pandas.DataFrame.from_dict(entry_data_upsampling_response)

In [ ]:
timeSeriePlot(entry_data_upsampling_response,param_name,label,title)

<ul><li>Downsampling</li></ul>

Let's image we want just one value per day instead: i.e the daily mean!

In [ ]:
how, rule = 'mean', '1d'

In [ ]:
end_point = '/entries/'+entry_id
aux_endpoint = '/data/'
query_parameters = 'resample_how='+how+'&resample_rule='+rule+'&standard_variable='+standard_variable+'&initial_datetime='+oneweekago+'&end_datetime='+times[-1]
url = '%s%s%s?%s' % (api_url, end_point, aux_endpoint, query_parameters)
entry_data_downsampling_request= requests.get(url, headers=headers)
entry_data_downsampling_response = json.loads(entry_data_downsampling_request.text)

print('API query: '+ url)
print('Find next the preview:')
pandas.DataFrame.from_dict(entry_data_downsampling_response)

In [ ]:
timeSeriePlot(entry_data_downsampling_response,param_name,label,title)

---

## Next tutorial

<div class="alert alert-block alert-success" style="margin-left: 2em">
<b>More! - UNDERCONSTRUCTION...</b>
    
***  
Discover now SOCIB instruments trough the `instruments` dedicated notebooks:
<ul>
    <li><span><a href="03-instruments/01-instrument-details.ipynb">01-instrument-details</a></span></li>
    <li><span><a href="03-instruments/02-instrument-search.ipynb">02-instrument-search</a></span></li>
    <li><span><a href="03-instruments/03-instrument-composing-data-sources.ipynb">03-instrument-composing-data-sources</a></span></li>
    <li><span><a href="03-instruments/04-instrument-composing-entries.ipynb">04-instrument-composing-entries</a></span></li>    
</ul>